In [39]:
import numpy as np
import statistics
import numpy as np
import scipy.stats
import pandas as pd
import datetime as dt

### Данные по определению качества сварки швов. Примечательно, что практически все колонки являются категориальными признаками (Клеймо, Вид сварки, Диаметр и толщина стенки, Марка стали, Негоден). Колонку "Дата заявки" впоследствии преобразуется в "Год заявки". Колонка "Негоден" является категориальной и сразу же преобразовывается в бинарную и характеризуется, как наличие брака в сварочном шве - цифра 1 и отсутствие брака - цифра 0. 

In [40]:
df=pd.read_excel('сварка.xlsx')
df.head()

,п/п,клеймо,Вид сварки,Производитель работ,диаметр и толщина стенки,Марка стали,Заявка,Годен / не годен
0,1,2789,РД,другое,159х6,М01,2018-06-04,годен
1,2,2789,РАД,другое,другое,М01,2018-06-04,годен
2,5,2SKB,РД,другое,159х8,М01,2018-06-04,годен
3,6,2SKB,РД,другое,159х8,М01,2018-06-04,годен
4,7,2SKB,РАД,другое,другое,М01,2018-06-04,годен


In [41]:
df['Годен / не годен'] = df['Годен / не годен'].replace('годен', 0)
df['Годен / не годен'] = df['Годен / не годен'].replace('негоден', 1)
df['Годен / не годен'] = df['Годен / не годен'].astype(int)

df.columns=['n','kleimo','vid_svarki','prozvoditel','diam_tolsch_stenki','marka_stali','zayavka','negoden']
df['Year'] = df['zayavka'].dt.to_period('Y')

In [42]:
df.dtypes

n                              int64
kleimo                        object
vid_svarki                    object
prozvoditel                   object
diam_tolsch_stenki            object
marka_stali                   object
zayavka               datetime64[ns]
negoden                        int64
Year                   period[A-DEC]
dtype: object

In [43]:
df.describe()

,n,negoden
count,5088.000000,5088.000000
mean,952.466785,0.143475
std,631.649832,0.350591
min,1.000000,0.000000
25%,435.000000,0.000000
50%,860.000000,0.000000
75%,1405.000000,0.000000
max,2425.000000,1.000000


In [44]:
# Определение процента негодных швов
negoden = (df.negoden == 1).sum(axis=0)
round(negoden/df.shape[0]*100,1)

14.3

### ГИПОТЕЗА - Влияют ли признаки из датасета на показатель брака? Так как все данные являются категориальными, будем делать попарные сравнения признаков с целевой переменной с помощью критерия ХИ КВАДРАТ

In [45]:
# ХИ2.ТЕСТ для переменных «Годен / не годен» и «Вид сварки»
df_chi = pd.crosstab(df['negoden'], df['vid_svarki'])
df_chi

vid_svarki,РАД,РД,другое
negoden,,,
0,2286,1931,141
1,119,593,18


In [46]:
table = [ [ 2286, 1931, 141 ], [ 119, 593, 18 ] ]

chi2, prob, dof, expected = scipy.stats.chi2_contingency(table)

output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"

print(output.format( chi2, dof, prob))

print(expected)
print('\n')
if prob < 0.05:
    print("Cвязь есть, различия значимы, переменные зависимы друг от друга")
else:
    print("Cвязи нет,  различия незначимы, переменные независимы друг от друга")

test Statistics: 345.93069434041115
degrees of freedom: 2
p-value: 7.622618130316593e-76

[[2059.94300314 2161.86949686  136.1875    ]
 [ 345.05699686  362.13050314   22.8125    ]]


Cвязь есть, различия значимы, переменные зависимы друг от друга


In [47]:
# ХИ2.ТЕСТ для переменных «Годен / не годен» и «Производитель работ»
df_chi = pd.crosstab(df['negoden'], df['prozvoditel'])
df_chi

prozvoditel,БРАВ-ТР,ГСИ ВФ НЗМ,КитСтрой,СГ-Волга,СГМ,СМУ №7,другое
negoden,,,,,,,
0,246,466,409,251,147,426,2413
1,8,33,88,54,16,108,423


In [48]:
table = [ [ 246,466,409,251,147,426,2413 ], [ 8,33,88,54,16,108,423 ] ]

chi2, prob, dof, expected = scipy.stats.chi2_contingency(table)

output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"

print(output.format( chi2, dof, prob))

print(expected)
print('\n')
if prob < 0.05:
    print("Cвязь есть, различия значимы, переменные зависимы друг от друга")
else:
    print("Cвязи нет, различия незначимы, переменные независимы друг от друга")

test Statistics: 76.04472916650006
degrees of freedom: 6
p-value: 2.3387249708439886e-14

[[ 217.55738994  427.40605346  425.69300314  261.24017296  139.61360063
   457.38443396 2429.10534591]
 [  36.44261006   71.59394654   71.30699686   43.75982704   23.38639937
    76.61556604  406.89465409]]


Cвязь есть, различия значимы, переменные зависимы друг от друга


In [49]:
# ХИ2.ТЕСТ для переменных «Годен / не годен» и «Диаметр и толщина стенки»
df_chi = pd.crosstab(df['negoden'], df['diam_tolsch_stenki'])
df_chi

diam_tolsch_stenki,159х6,159х8,32х4,32х5,57х5,57х6,другое
negoden,,,,,,,
0,830,706,342,184,723,558,1015
1,230,206,52,27,67,28,120


In [50]:
table = [ [ 830,706,342,184,723,558,1015 ], [ 230,206,52,27,67,28,120 ] ]

chi2, prob, dof, expected = scipy.stats.chi2_contingency(table)

output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"

print(output.format( chi2, dof, prob))

print(expected)
print('\n')
if prob < 0.05:
    print("Cвязь есть, различия значимы, переменные зависимы друг от друга")
else:
    print("Cвязи нет, различия не значимы, переменные независимы друг от друга")

test Statistics: 176.78466658436048
degrees of freedom: 6
p-value: 1.63426433889201e-35

[[907.91666667 781.1509434  337.47091195 180.72680818 676.65487421
  501.92374214 972.15605346]
 [152.08333333 130.8490566   56.52908805  30.27319182 113.34512579
   84.07625786 162.84394654]]


Cвязь есть, различия значимы, переменные зависимы друг от друга


In [51]:
# ХИ2.ТЕСТ для переменных «Годен / не годен» и «Марка стали»
df_chi = pd.crosstab(df['negoden'], df['marka_stali'])
df_chi

marka_stali,М01,М11,другое
negoden,,,
0,3632,462,264
1,666,31,33


In [52]:
table = [ [ 3632,462,264 ], [ 666,31,33 ] ]

chi2, prob, dof, expected = scipy.stats.chi2_contingency(table)

output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"

print(output.format( chi2, dof, prob))

print(expected)
print('\n')
if prob < 0.05:
    print("Cвязь есть, различия значимы, переменные зависимы друг от друга")
else:
    print("Cвязи нет, различия не значимы, переменные независимы друг от друга")

test Statistics: 33.19949661905186
degrees of freedom: 2
p-value: 6.177615987023092e-08

[[3681.34512579  422.26690252  254.3879717 ]
 [ 616.65487421   70.73309748   42.6120283 ]]


Cвязь есть, различия значимы, переменные зависимы друг от друга


In [53]:
# ХИ2.ТЕСТ для переменных «Годен / не годен» и «Год»
df_chi = pd.crosstab(df['negoden'], df['Year'])
df_chi

Year,2018,2019,2020
negoden,,,
0,845,2107,1406
1,163,313,254


In [54]:
table = [ [ 845,2107,1406 ], [ 163,313,254 ] ]

chi2, prob, dof, expected = scipy.stats.chi2_contingency(table)

output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"

print(output.format( chi2, dof, prob))

print(expected)
print('\n')
if prob < 0.05:
    print("Cвязь есть, различия значимы, переменные зависимы друг от друга")
else:
    print("Cвязи нет, различия не значимы, переменные независимы друг от друга")

test Statistics: 7.890139751045273
degrees of freedom: 2
p-value: 0.01934986424056103

[[ 863.37735849 2072.7908805  1421.83176101]
 [ 144.62264151  347.2091195   238.16823899]]


Cвязь есть, различия значимы, переменные зависимы друг от друга


### Все признаки в разной степени влияют на целевой показатель (наличие брака в сварочном шве), во всех тестах ХИ квадрат есть значимые различия, значения p < 0.05